In [ ]:
!pip -qq install catboost

     |████████████████████████████████| 69.2MB 91kB/s 


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

In [3]:
!gdown --id 1SKv5wNfLo4MCAI2Yao7QIM6ZXQ8RXdDU
!unzip -q /content/silicon_data.zip

train = pd.read_csv('/content/silicon_data/Train.csv')
test = pd.read_csv('/content/silicon_data/Test.csv')
sample = pd.read_csv('/content/silicon_data/SampleSubmission.csv')

Downloading...
From: https://drive.google.com/uc?id=1SKv5wNfLo4MCAI2Yao7QIM6ZXQ8RXdDU
To: /content/silicon_data.zip
100% 667k/667k [00:00<00:00, 98.1MB/s]


In [4]:
train['total'] = train.percentage_primary_schooling+train.percentage_secondary_schooling+train.percentage_tertiary_schooling
test['total'] = test.percentage_primary_schooling+test.percentage_secondary_schooling+test.percentage_tertiary_schooling
cols = ['percentage_no_schooling', 'literate_pop', 'total']

X = train[cols]
y = train.target
test_df = test[cols]

In [5]:
%%time
xgb, rf = [], []
for i,( train_index, test_index) in enumerate(KFold(10).split(X, y)):
    X_tra, X_val, y_tra, y_val = X.iloc[train_index], X.iloc[test_index], y[train_index], y[test_index]

    model = XGBRegressor(objective = 'reg:squarederror', eval_metric = 'mae', n_estimators=10000)
    model.fit(X_tra, y_tra, eval_set = [(X_tra, y_tra), (X_val, y_val)], early_stopping_rounds=150, verbose = False)
    xgb.append(model.predict(test_df))

    model = RandomForestRegressor(random_state=100, min_samples_split=2)
    model.fit(X_tra, y_tra)
    rf.append(model.predict(test_df))

sub = pd.DataFrame({'ID': test.ID, 'target': np.mean([np.mean(xgb, 0), np.mean(rf, 0)], 0)})
sub.to_csv('Final_sub.csv', index = False)

CPU times: user 7min 37s, sys: 1.18 s, total: 7min 38s
Wall time: 7min 37s
